In [1]:
import numpy as np
import pandas as pd
import os
import collections
import re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, PorterStemmer, SnowballStemmer, WordNetLemmatizer
import time
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cdist
import pickle

# 1. Helper functions

In [2]:
def textNormalize(rawString):
    """
    Function for text normalization.
    Text normalization includes:
    1. removing web links
    2. converting all letters to lower or upper case
    3. removing punctuations
    4. removing numbers
    5. tokenization
    6. removing stopwords
    7. stemming
    8. lemmatization
    Input:
        rawString: a string containing text to be normaized. 
    Output:
        normText: a string containing the normalized text where the tokens extracted from rawString are joined by space.
    """
    if rawString == np.nan:
        return rawString
    ## Remove web links
    rawString = re.sub('https?://\S+|www\.\S+', '', rawString) 

    ## Lowercase
    rawString = rawString.lower()
    
    ## Remove punctuation
    rawString = re.sub('<.*?>+', ' ', rawString)
    rawString = re.sub('[%s]' % re.escape(string.punctuation), ' ', rawString)
    
    ## Remove number
    rawString = re.sub(r'\d+', '', rawString)
    
    ## Tokenize
    words = word_tokenize(rawString)
    
    ## Remove stop words
    nltk_stop_words = stopwords.words('english')
    words = [word for word in words if word not in nltk_stop_words]
    
    ## Stem
    stemmer = SnowballStemmer('english')
    words = [stemmer.stem(word) for word in words]
    
    ## Lematize verbs
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word, pos='v') for word in words]
    
    normText = " ".join(words)
    
    return normText

# 2. Preprocess metadata

The preprocessing step only needs to be run once. The new metadata table with the normalized abstracts in the column "clean_abstract" will be saved to all_sources_metadata_2020-03-13_clean.csv

## 2.1. Read-in metadata

In [3]:
#metaDataPath = "/kaggle/input/CORD-19-research-challenge/2020-03-13/all_sources_metadata_2020-03-13.csv"
metaDataPath = "~/Downloads/2020-03-13/all_sources_metadata_2020-03-13.csv"
metaData = pd.read_csv(metaDataPath, header = 0, index_col = 0)
print("The number of literatures: " + str(metaData.shape[0]))
metaData.head()

The number of literatures: 29500


,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text
sha,,,,,,,,,,,,,
c630ebcdf30652f0422c3ec12a00b50241dc9bd9,CZI,Angiotensin-converting enzyme 2 (ACE2) as a SA...,10.1007/s00134-020-05985-9,NaN,32125455.0,cc-by-nc,NaN,2020,"Zhang, Haibo; Penninger, Josef M.; Li, Yimin; ...",Intensive Care Med,2.002765e+09,#3252,True
53eccda7977a31e3d0f565c884da036b1e85438e,CZI,Comparative genetic analysis of the novel coro...,10.1038/s41421-020-0147-1,NaN,NaN,cc-by,NaN,2020,"Cao, Yanan; Li, Lin; Feng, Zhimin; Wan, Shengq...",Cell Discovery,3.003431e+09,#1861,True
210a892deb1c61577f6fba58505fd65356ce6636,CZI,Incubation Period and Other Epidemiological Ch...,10.3390/jcm9020538,NaN,NaN,cc-by,The geographic spread of 2019 novel coronaviru...,2020,"Linton, M. Natalie; Kobayashi, Tetsuro; Yang, ...",Journal of Clinical Medicine,3.006065e+09,#1043,True
e3b40cc8e0e137c416b4a2273a4dca94ae8178cc,CZI,Characteristics of and Public Health Responses...,10.3390/jcm9020575,NaN,32093211.0,cc-by,"In December 2019, cases of unidentified pneumo...",2020,"Deng, Sheng-Qun; Peng, Hong-Juan",J Clin Med,1.776631e+08,#1999,True
92c2c9839304b4f2bc1276d41b1aa885d8b364fd,CZI,Imaging changes in severe COVID-19 pneumonia,10.1007/s00134-020-05976-w,NaN,32125453.0,cc-by-nc,NaN,2020,"Zhang, Wei",Intensive Care Med,3.006643e+09,#3242,False


## 2.2. Normalize abstracts

In [4]:
startTime = time.time()
metaData["clean_abstract"] = float("NaN")
metaData.loc[metaData["abstract"].notnull(), "clean_abstract"] = \
metaData["abstract"][metaData["abstract"].notnull()].apply(lambda x: textNormalize(x))
print("Time spent: " + str(round((time.time() - startTime) / 60, 3)) + "min.")

Time spent: 2.231min.


## 2.3. Output the new metadata table

In [5]:
metaData.to_csv("~/Downloads/2020-03-13/all_sources_metadata_2020-03-13_clean.csv")

# 3. tf-idf vectorizer

This step only needs to be run once. A tf-idf vectorizer will be trained from the normalized abstracts and saved to tf-idf_vectorizer.pkl

## 3.1. Read-in metadata with normalized abstracts

In [2]:
metaDataPath = "~/Downloads/2020-03-13/all_sources_metadata_2020-03-13_clean.csv"
metaData = pd.read_csv(metaDataPath, header = 0, index_col = 0)
metaData.head()

,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,clean_abstract
sha,,,,,,,,,,,,,,
c630ebcdf30652f0422c3ec12a00b50241dc9bd9,CZI,Angiotensin-converting enzyme 2 (ACE2) as a SA...,10.1007/s00134-020-05985-9,NaN,32125455.0,cc-by-nc,NaN,2020,"Zhang, Haibo; Penninger, Josef M.; Li, Yimin; ...",Intensive Care Med,2.002765e+09,#3252,True,NaN
53eccda7977a31e3d0f565c884da036b1e85438e,CZI,Comparative genetic analysis of the novel coro...,10.1038/s41421-020-0147-1,NaN,NaN,cc-by,NaN,2020,"Cao, Yanan; Li, Lin; Feng, Zhimin; Wan, Shengq...",Cell Discovery,3.003431e+09,#1861,True,NaN
210a892deb1c61577f6fba58505fd65356ce6636,CZI,Incubation Period and Other Epidemiological Ch...,10.3390/jcm9020538,NaN,NaN,cc-by,The geographic spread of 2019 novel coronaviru...,2020,"Linton, M. Natalie; Kobayashi, Tetsuro; Yang, ...",Journal of Clinical Medicine,3.006065e+09,#1043,True,geograph spread novel coronavirus covid infect...
e3b40cc8e0e137c416b4a2273a4dca94ae8178cc,CZI,Characteristics of and Public Health Responses...,10.3390/jcm9020575,NaN,32093211.0,cc-by,"In December 2019, cases of unidentified pneumo...",2020,"Deng, Sheng-Qun; Peng, Hong-Juan",J Clin Med,1.776631e+08,#1999,True,decemb case unidentifi pneumonia histori expos...
92c2c9839304b4f2bc1276d41b1aa885d8b364fd,CZI,Imaging changes in severe COVID-19 pneumonia,10.1007/s00134-020-05976-w,NaN,32125453.0,cc-by-nc,NaN,2020,"Zhang, Wei",Intensive Care Med,3.006643e+09,#3242,False,NaN


## 3.2.	Learn vocabulary and idf from training set

In [3]:
startTime = time.time()
vectorizer = TfidfVectorizer(tokenizer = word_tokenize)
vectorizer.fit(metaData["clean_abstract"][metaData["clean_abstract"].notnull()])
print('Time spent: ' + str(time.time() - startTime) + 's.')

Time spent: 20.281893014907837s.


## 3.3. Save the learned vectorizer

In [4]:
vectPath = "../../../Downloads/2020-03-13/tf-idf_vectorizer.pkl"
with open(vectPath, "wb") as vectFile:
    pickle.dump(vectorizer, vectFile)

# 4. Retrieve literatures for each task

## 4.1. Read-in metadata with normalized abstracts

In [3]:
metaDataPath = "~/Downloads/2020-03-13/all_sources_metadata_2020-03-13_clean.csv"
metaData = pd.read_csv(metaDataPath, header = 0, index_col = 0)
metaData.head()

,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,clean_abstract
sha,,,,,,,,,,,,,,
c630ebcdf30652f0422c3ec12a00b50241dc9bd9,CZI,Angiotensin-converting enzyme 2 (ACE2) as a SA...,10.1007/s00134-020-05985-9,NaN,32125455.0,cc-by-nc,NaN,2020,"Zhang, Haibo; Penninger, Josef M.; Li, Yimin; ...",Intensive Care Med,2.002765e+09,#3252,True,NaN
53eccda7977a31e3d0f565c884da036b1e85438e,CZI,Comparative genetic analysis of the novel coro...,10.1038/s41421-020-0147-1,NaN,NaN,cc-by,NaN,2020,"Cao, Yanan; Li, Lin; Feng, Zhimin; Wan, Shengq...",Cell Discovery,3.003431e+09,#1861,True,NaN
210a892deb1c61577f6fba58505fd65356ce6636,CZI,Incubation Period and Other Epidemiological Ch...,10.3390/jcm9020538,NaN,NaN,cc-by,The geographic spread of 2019 novel coronaviru...,2020,"Linton, M. Natalie; Kobayashi, Tetsuro; Yang, ...",Journal of Clinical Medicine,3.006065e+09,#1043,True,geograph spread novel coronavirus covid infect...
e3b40cc8e0e137c416b4a2273a4dca94ae8178cc,CZI,Characteristics of and Public Health Responses...,10.3390/jcm9020575,NaN,32093211.0,cc-by,"In December 2019, cases of unidentified pneumo...",2020,"Deng, Sheng-Qun; Peng, Hong-Juan",J Clin Med,1.776631e+08,#1999,True,decemb case unidentifi pneumonia histori expos...
92c2c9839304b4f2bc1276d41b1aa885d8b364fd,CZI,Imaging changes in severe COVID-19 pneumonia,10.1007/s00134-020-05976-w,NaN,32125453.0,cc-by-nc,NaN,2020,"Zhang, Wei",Intensive Care Med,3.006643e+09,#3242,False,NaN


## 4.2. Load the tf-idf vectorizer

In [4]:
vectPath = "../../../Downloads/2020-03-13/tf-idf_vectorizer.pkl"
with open(vectPath, 'rb') as vectFile:
    vectorizer = pickle.load(vectFile)

## 4.3. Transform document to document-term matrix

In [5]:
docTermMatrix = vectorizer.transform(metaData["clean_abstract"][metaData["clean_abstract"].notnull()]).toarray()

## 4.4. Task I: What is known about transmission, incubation, and environmental stability?

In [12]:
queries = ["transmission incubation and environmental stability"]

In [14]:
queries = [textNormalize(q) for q in queries]
queryTermMatrix = vectorizer.transform(queries).toarray()

In [15]:
queryDocDist = 1 - cdist(queryTermMatrix, docTermMatrix, metric = 'cosine')

In [16]:
for i in range(queryDocDist.shape[0]):
    print(queryDocDist[i].argsort()[-3:][::-1])

[  780 18358 23542]


In [78]:
metaData["abstract"][metaData["abstract"].notnull()][129]

'Coronaviruses have been implicated in nosocomial outbreaks with environmental contamination as a route of transmission. Similarly, nosocomial transmission of severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) has been reported. However, the mode of transmission and extent of environmental contamination are unknown.'